# コピー文 前処理

In [5]:
import pandas as pd
pd.set_option("display.max_colwidth", 500)
pd.set_option("display.max_rows", 101)
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
gs_table_url = 'gs://wakimoto-ramiel/experiments/exp_20191220_kw2title/table/cosme.tsv'
data_path = './data/tmp/cosme.tsv'
! gsutil cp $gs_table_url $data_path

Copying gs://wakimoto-ramiel/experiments/exp_20191220_kw2title/table/cosme.tsv...
- [1 files][ 67.4 MiB/ 67.4 MiB]                                                
Operation completed over 1 objects/67.4 MiB.                                     


In [64]:
df = pd.read_csv(data_path, sep='\t')

## 単体テスト用

In [7]:
import sys
import logging
logging.basicConfig(level=logging.DEBUG)
def unit_test(test_case_list):
    # 単体テスト. in/outが1変数のやつしか対応していない
    def _unit_test(func):
        for test_case in test_case_list:
            _in, _out, _comment = None, None, None
            if len(test_case) == 2:
                _in, _out = test_case
            elif len(test_case) == 3:
                _in, _out, _comment = test_case
            else:
                raise Exception(f'{test_case}の長さは2か3であるべきです. テストケースを書き直してください.')
            res = func(_in)
            assert res == _out, f'''
Error :{_comment}
Input :{_in}
Label :{_out}
Output:{res}
'''
        logging.info(f'【unit_test】 {func.__name__} は{len(test_case_list)}件のテストに通りました.')
        return func
    return _unit_test

## Mecab 前処理
- URL除去
- ハッシュタグ除去
- コロン削除： 30,000等の形態素解析でノイズなので
- 文末, 文頭の空白削除
- 文中の全角空白は「。」にする
- カタカナ記号は全角で統一. 数字,記号の正規化
- *1 とかの注釈マークを除去
- TODO: 顔文字をトークンとしてみなす

In [48]:
# df_uq_text[df_uq_text.text.str.contains('　')].sample(100)

In [121]:
import re
import jaconv
SEP_TAG = '<sep>'
JA_PERIOD = '。'
unit_test_success_log = {}

def is_valid_sentence(text):
    return bool(text)

def before_tagger_processing(text):
    # 形態素解析にかける前に行う前処理
    # 空白除去など
    
    # URL削除
    text = remove_url(text)
    
    # ハッシュタグ削除
    text = remove_hash(text)
    
    # コロン削除
    # - 本当は数詞内のみ削除, 他は<sep>にしたい
    text = remove_coron(text)
    
    # 文末, 文頭の空白削除
    text = remove_head_tail_space(text)
    
    # 文中の全角空白は「。」にする
    text = replace_fwspace_ja_period(text)
    
    # ----- カタカナ記号は全角で統一. 数字,記号の正規化 ---------
    text = normalize(text)
    text = re.sub('\s', '', text)
    
    # 「。!?」は<sep>をつける：これはmecab後のやつ
    # text = replace_ja_period_sep(text)
    
    # *1 とかの注釈マークを除去
    text = remove_notes_mark(text)
    
    return text

@unit_test([
    ('7日間のエイジングケア(*1)で集中アプローチ。高濃度イモーテル(*2)の力で素肌にハリ*1。', '7日間のエイジングケアで集中アプローチ。高濃度イモーテルの力で素肌にハリ。'),
    ('ああ*1これ*2ああ(*2)でも(*)だけど(*3)', 'ああこれああでもだけど'),
    ('ああ※1これ※2ああ(※2)でも(※)だけど(※3)', 'ああこれああでもだけど'),
    ('こういうのは微妙(*hoge), データ(*hoge)にないからいい', 'こういうのは微妙(hoge), データ(hoge)にないからいい'),
])
def remove_notes_mark(text):
    # *, *1, (*2), (*)といった注釈マーク消す. ※も.
    text = re.sub(r'\([\*|※]\d*\)', '', text)
    text = re.sub(r'[\*|※]\d*', '', text)
    return text


@unit_test([
    ('ｼﾘｺﾝﾊﾟﾌから､ｸｯｼｮﾝﾌｧﾝﾃﾞ専用､新登場｡', 'シリコンパフから、クッションファンデ専用、新登場。', 'カタカナは全角に'),
    ('１０００, 1000', '1000, 1000', '数字は半角に'),
    ('!!!???***, ！！！？？？＊＊＊', '!!!???***, !!!???***', '記号は半角に'),
    ('"これはそのまま"', '"これはそのまま"', '「"」はそのままにする'),
])
def normalize(text):
    # カタカナは全角に
    # ~,_などの正規化, 数字, 記号は半角にしてくれる
    # "hoge" を ``hoge"にするのは期待に沿わないので, "hoge"に直す
    text = jaconv.normalize(text, 'NFKC')
    text = re.sub(r'``', '"', text)
    return text

@unit_test([
    ('いつでもどこでも、すうっと一息。アロマのマッサージ。', 'いつでもどこでも、すうっと一息。<sep>アロマのマッサージ。', '<sep>を入れる. 文末にはいれない'),
    ('ああ!!??なに?これ???', 'ああ!!??<sep>なに?<sep>これ???', '記号が混ざっても複数<sep>を入れない'),
])
def replace_ja_period_sep(text):
    # 。や!の後に<sep>を挿入. 。とか!は効果や表現に意味を持ちそうなので今回除去しない
    text = re.sub(r'([！|？|!|?|。]+)', r'\1<sep>', text)
    text = re.sub(r'<sep>$', '', text)
    return text

@unit_test([
    ('うねりやすい髪が気になる方に　すんなりまとまる髪へ', f'うねりやすい髪が気になる方に{JA_PERIOD}すんなりまとまる髪へ'),
    ('うねりやすい髪が気になる方に　　すんなりまとまる髪へ', f'うねりやすい髪が気になる方に{JA_PERIOD}すんなりまとまる髪へ', '複数あったらまとめて置換'),
    ('いつでもどこでも うるおいケア', f'いつでもどこでも うるおいケア', '半角は対象としない'),
])
def replace_fwspace_ja_period(text):
    # 文中の全角空白は「。」に（文末, 文頭の空白は除去されている前提）
    return re.sub(r'　+', JA_PERIOD, text)

@unit_test([
    ('肌をなめらかにほぐす,柔らか化粧水', '肌をなめらかにほぐす柔らか化粧水'),
])
def remove_coron(text):
    # コロン削除
    return re.sub(r',', '', text)

@unit_test([
    ('  aaa','aaa', '文頭の空白は消す'),
    ('aaa   ','aaa', '文末の空白は消す'),
    ('  aaa　　','aaa', '文頭,文末の空白は消す. 半角全角は区別しない.'),
    ('　　aaa','aaa', '半角全角は区別しない'),
    ('　　a   aa　　　','a   aa', '文中の空白は消さない'),
    ('a  aa','a  aa', '文中の空白は消さない'),
])
def remove_head_tail_space(text):
    # 文頭, 文末の空白削除
    text = re.sub(r'\s+$', '', text)
    text = re.sub(r'^\s+', '', text)
    return text

@unit_test([
    ('https://hoge.com',''),
    ('hogehttps://hoge.com だね','hoge だね'),
    ('↓少しでも気になる方はまずクリック↓ https://hoge.com','↓少しでも気になる方はまずクリック↓ '),
])
def remove_url(text):
    return re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', text)

@unit_test([
    ('おすすめです!#HOGE #SAIKO だよ','おすすめです!  だよ'),
])
def remove_hash(text):
    return re.sub(r'[#＃][Ａ-Ｚａ-ｚA-Za-z一-鿆0-9０-９ぁ-ヶｦ-ﾟー]+', '', text)

INFO:root:【unit_test】 remove_notes_mark は4件のテストに通りました.
INFO:root:【unit_test】 normalize は4件のテストに通りました.
INFO:root:【unit_test】 replace_ja_period_sep は2件のテストに通りました.
INFO:root:【unit_test】 replace_fwspace_ja_period は3件のテストに通りました.
INFO:root:【unit_test】 remove_coron は1件のテストに通りました.
INFO:root:【unit_test】 remove_head_tail_space は6件のテストに通りました.
INFO:root:【unit_test】 remove_url は3件のテストに通りました.
INFO:root:【unit_test】 remove_hash は1件のテストに通りました.


## Mecab：分かち書き

In [209]:
import MeCab
from functools import lru_cache
m = MeCab.Tagger("-Ochasen -d /usr/lib/mecab/dic/mecab-ipadic-neologd")

@lru_cache(maxsize = None)
def mecab_parse(text):
    """
    textをmecabでパースする
    input: 
    """
    parsed_list = []
    text_parsed = m.parse(text)
    for word_parsed in text_parsed.split('\n')[:-2]: # EOF省略
        wp_list = word_parsed.split('\t')
        origin, kana, stem, pos_list, _, _ = wp_list
        parsed_list.append({
            'origin': origin,
            'kana': kana,
            'stem': stem,
            'pos_list': pos_list,
        })
    return parsed_list

def wakati(text):
    if type(text) is str:
        tag_list = parse_text(text)
    elif type(text) is list and text and 'origin' in text[0].keys():
        tag_list = text
    text = ' '.join([tag['origin'] for tag in tag_list])
    return text

def contains_only_noun(pos_list):
    # 名詞しか使ってなかったら非文 : 368件しかなかったのでやめとく
    return sum(['名詞' in pos for pos in pos_list]) == len(pos_list)

def is_valid_sentence(tag_list):
    origin = ''.join([tag['origin'] for tag in tag_list])
    pos_list = [tag['pos_list'] for tag in tag_list]
    # if contains_only_noun(pos_list): return False
    # ひらがな, カタカナ以外のみで文が構成されているか
    is_only_contains_not_kana = bool(re.match(r'^[^ぁ-ん^ァ-ン]*$', origin))
    return not is_only_contains_not_kana

In [210]:
# mecab_parse('ピコ太郎さんはカナブンに角をつけてカブトムシとして売るバイトをしている')

## Mecab後処理

特別な数字トークン以外の数字を<num>にする  
。！？ の後に<sep>を入れる

In [260]:
# 他の数字とは明らかに文表現が異なる単語リスト
num_special_words = ['1本', '1個', '100%', '1日', '1品', '365日', '1つ', '1枚', '24時間', '3D', 'No.1', '99%', '1度']

def after_tagger_processing(text):
    # <sep>の挿入
    text = replace_ja_period_sep_wakatied(text)
    # 数詞トークンをマスク
    text = mask_general_num_token_wakatied(text)
    return text

@unit_test([
    ('お 得 な 3つ で 2 役 約120g 。 100% です 。 2.0 立法 。', 'お 得 な <num> つ で <num> 役 約 <num> g 。 100% です 。 <num> 立法 。', 'マスクする'),
    ('1 2ml 入ってます', '<num> ml 入ってます', '<num>を連続させない'),
])
def mask_general_num_token_wakatied(text):
    # 一般的な数詞トークンをマスク, 分割
    word_list = text.split(' ')
    masked_word_list = []
    for word in word_list:
        # 特殊数字ならそのまま
        if word in num_special_words:
            masked_word_list.append(word)
            continue
            
        # それ以外の普遍的な数字はマスクし前後に空白を挟む
        word = re.sub(r'(?:\d+\.?\d*|\.\d+)', r' <num> ', word)
        splited_word_list = [w for w in word.split(' ') if w]
        masked_word_list += splited_word_list
    masked_word_list = unite_duplicate(masked_word_list, '<num>')
    masked_word_list = unite_duplicate(masked_word_list, '<sep>')
    return ' '.join(masked_word_list)

@unit_test([
    ('これ だ ! それ だ 。 どれ だ ？ これ だ ！', 'これ だ ! <sep> それ だ 。 <sep> どれ だ ？ <sep> これ だ ！', '<sep>を入れる. 文末にはいれない'),
    ('これ だ !!!! それ だ !!???。。 これ だ !', 'これ だ !!!! <sep> それ だ !!???。。 <sep> これ だ !', '記号が混ざっても複数<sep>を入れない'),
])
def replace_ja_period_sep_wakatied(text):
    # 。や!の後に<sep>を挿入. 。とか!は効果や表現に意味を持ちそうなので今回除去しない
    # mecab後のやつ. 空白で挟む
    text = re.sub(r'([！|？|!|?|。]+)', r'\1 <sep>', text)
    text = re.sub(r' *<sep>$', '', text)
    return text

def unite_duplicate(arr, unit):
    # 連続するunit要素をまとめて1個にする
    new_arr = []
    pre_item = None
    for item in arr:
        if item == pre_item and unit == item:
            pre_item = item
            continue
        else:
            new_arr.append(item)
            pre_item = item
    return new_arr

INFO:root:【unit_test】 mask_general_num_token_wakatied は1件のテストに通りました.
INFO:root:【unit_test】 replace_ja_period_sep_wakatied は2件のテストに通りました.


In [259]:
unite_duplicate([0, 1, 1, 0, 0, 2, 2, 1, 2, 2], 1)

[0, 1, 0, 0, 2, 2, 1, 2, 2]

## Main処理

In [66]:
# df[['text', 'media', 'parsed']]

In [277]:
df = df[['text', 'media', 'parsed']]

In [278]:
df = df[~df.text.isna()].copy()
print(f'Nan除去：{df.shape[0]}件')

Nan除去：192042件


In [279]:
df_uq_text = df.drop_duplicates('text').copy()
print(f'重複除去：{df_uq_text.shape[0]}件')

重複除去：50639件


In [280]:
%%time
# わかちがき前処理
# - コロン削除 ： 30,000とかのため
# - 文末, 文頭の空白削除
# - 文中の全角空白は「。」にする
# - カタカナ記号は全角で統一. 数字,記号の正規化
# - *1 とかの注釈マークを除去
# - URL除去
# - ハッシュタグ除去
df_uq_text['normalized_text'] = df_uq_text.text.apply(before_tagger_processing)

CPU times: user 1.26 s, sys: 4 ms, total: 1.26 s
Wall time: 1.28 s


In [281]:
%%time
# タグづけ
df_uq_text['mecab_tagged_text'] = df_uq_text.normalized_text.apply(mecab_parse)

CPU times: user 36 ms, sys: 0 ns, total: 36 ms
Wall time: 36.1 ms


In [282]:
%%time
# POSタグ抽出 ： 一旦愚直に並べてみる
df_uq_text['pos_all'] = df_uq_text.mecab_tagged_text.apply(lambda tags: [t['pos_list'] for t in tags])
df_uq_text['pos_large'] = df_uq_text.mecab_tagged_text.apply(lambda tags: [t['pos_list'].split('-')[0] for t in tags])

CPU times: user 1.28 s, sys: 72 ms, total: 1.35 s
Wall time: 1.36 s


In [283]:
%%time
# 分かち書き
df_uq_text['wakatied_text'] = df_uq_text.mecab_tagged_text.apply(wakati)

CPU times: user 156 ms, sys: 4 ms, total: 160 ms
Wall time: 163 ms


In [284]:
%%time
# わかちがき後処理
df_uq_text['complete_processed_text'] = df_uq_text.wakatied_text.apply(after_tagger_processing)

CPU times: user 2.33 s, sys: 12 ms, total: 2.34 s
Wall time: 2.36 s


In [285]:
%%time
# 非文除去
df_uq_text['is_valid'] = df_uq_text.mecab_tagged_text.apply(is_valid_sentence)
print(f'全レコード: {df_uq_text.shape[0]}件')
df_uq_valid_text = df_uq_text[df_uq_text.is_valid].copy()
print(f'非文除去後: {df_uq_valid_text.shape[0]}件')

全レコード: 50639件
非文除去後: 50362件
CPU times: user 288 ms, sys: 4 ms, total: 292 ms
Wall time: 297 ms


In [365]:
# POSチェック
# df_uq_valid_text['pos_large_str'] = df_uq_valid_text.pos_large.apply(lambda pos_list: str(pos_list))
# print('<br/>'.join(df_uq_valid_text[df_uq_valid_text.pos_large_str == "['名詞', '助詞', '名詞', '記号', '名詞', '助詞', '動詞', '名詞', '助詞']"].text.tolist()[:5]))

In [353]:
# 整形後の重複除去
df_uq_valid_text = df_uq_valid_text.drop_duplicates('complete_processed_text')

# データ保存

In [428]:
df_uq_valid_text = df_uq_valid_text[['text', 'media', 'normalized_text', 'complete_processed_text', 'mecab_tagged_text', 'pos_all', 'pos_large']].copy()
df_uq_valid_text.to_pickle('./data/tmp/df_cosme_parsed.pickle')

# 学習用データ分割,吐き出し

In [438]:
DATA_TYPES = ['train', 'valid', 'test']

In [439]:
def split_df(df, train_rate=0.8):
    # データシャッフル
    df = df.sample(frac=1, random_state=199).reset_index(drop=True)
    df_size = df.shape[0]
    train_index = int(df_size * 0.9)
    valid_index = int(train_index + df_size * 0.05)
    df['data_type'] = None
    df['data_type'][:train_index] = 'train'
    df['data_type'][train_index:valid_index] = 'valid'
    df['data_type'][valid_index:] = 'test'
    return df

In [440]:
df = split_df(df_uq_valid_text)

In [441]:
df.groupby('data_type').count()[['text']].T[DATA_TYPES]

data_type,train,valid,test
text,44596,2477,2479


In [442]:
def save_lines(lines, path):
    with open(path, 'w') as f:
        f.write('\n'.join(lines))
        print(f'save → {path}')

In [443]:
df['pos_str'] = df.pos_large.apply(lambda l: ' '.join(l))

In [481]:
def save_data_df_train(df):
    for data_type in DATA_TYPES:
        _df = df[df.data_type == data_type]
        save_lines(_df.complete_processed_text.tolist(), f'./data/tmp/{data_type}_lines.txt')
        save_lines(_df.pos_str.tolist(), f'./data/tmp/{data_type}_pos.txt')

In [482]:
save_data_df_train(df)

save → ./data/tmp/train_lines.txt
save → ./data/tmp/train_pos.txt
save → ./data/tmp/valid_lines.txt
save → ./data/tmp/valid_pos.txt
save → ./data/tmp/test_lines.txt
save → ./data/tmp/test_pos.txt


In [ ]:
# kao_reg = r'\([^あ-ん\u30A1-\u30F4\u2E80-\u2FDF\u3005-\u3007\u3400-\u4DBF\u4E00-\u9FFF\uF900-\uFAFF\U00020000-\U0002EBEF]+?\)'
# kao_reg = r'\(.*\)'
# df[df.normalized_text.str.match(kao_reg)][['text', 'complete_processed_text']]
# df[df.text.str.contains('\(')][['text', 'complete_processed_text']]
# df[df.text.str.contains('\(')].shape[0], df[df.text.str.contains('\（')].shape[0]
# df[df.normalized_text.str.contains('\(')][['text', 'complete_processed_text']]

# データアップロード

In [484]:
! gsutil -m rsync ./data/tmp gs://kawamoto-ramiel/experiments_v3_pos_20200104/data

Building synchronization state...
Starting synchronization...
Copying file://./data/tmp/cosme.tsv [Content-Type=text/tab-separated-values]...
Copying file://./data/tmp/df_cosme_parsed.pickle [Content-Type=application/octet-stream]...
Copying file://./data/tmp/test_lines.txt [Content-Type=text/plain]...
Copying file://./data/tmp/test_pos.txt [Content-Type=text/plain]...             
Copying file://./data/tmp/train_lines.txt [Content-Type=text/plain]...          
Copying file://./data/tmp/train_pos.txt [Content-Type=text/plain]...            
Copying file://./data/tmp/valid_lines.txt [Content-Type=text/plain]...          
Copying file://./data/tmp/valid_pos.txt [Content-Type=text/plain]...            
/ [8/8 files][150.8 MiB/150.8 MiB] 100% Done                                    
Operation completed over 8 objects/150.8 MiB.                                    
